In [30]:
import pandas as pd
import re
from datasets import Dataset

Load Dataset

In [10]:
data_path = '../data/naruto.csv'
naruto_transcript_df = pd.read_csv(data_path)

In [11]:
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


Removing actions from the trasnscript

In [12]:
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)
    return result

In [13]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [14]:
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [ ]:
naruto_transcript_df['number_of_words'] = naruto_transcript_df['line'].apply(lambda x: len(x.split()))

In [18]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",25
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [21]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name']=="Naruto")&(naruto_transcript_df['number_of_words']>5), 'naruto_response_flag'] = 1

In [22]:
naruto_transcript_df

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",25,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,27,0
159,Iruka,Huh?,1,0
160,Naruto,Iruka Sensei!,2,0
161,Iruka,Ah! That hurts!,3,0


In [23]:
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag'] == 1)&(naruto_transcript_df.index>0)].index)

In [24]:
indexes_to_take[:3]

[6, 28, 30]

In [25]:
system_promt = """ You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \n"""

prompys = []

for ind in indexes_to_take:
    naruto_response = naruto_transcript_df.loc[ind, 'line']
    naruto_response = naruto_response.replace('\n', '')
    naruto_response = naruto_response.replace('Naruto:', '')
    naruto_response = naruto_response.strip()
    prompt = system_promt + f"Naruto: {naruto_response}"
    prompys.append(prompt)

In [26]:
prompys

[' You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \nNaruto: Where’d you come from, Iruka Sensei!? What are you doing here?',
 ' You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \nNaruto: So what? It’s not like there’s anyone waiting at home for me.',
 ' You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \nNaruto: What do you want now, Sensei?',
 ' You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \nNaruto: Now that’s some serious motivation! I’ll have this clean in no time!',
 ' You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \nNaruto: Course I do.  Everybody knows. They were the greatest Shinobi of their time, right? The best of the best. Undefeated Ninja champs.   And the Fourth Hokage’s the one who saved the

I am doing a second approach where I will feed the prompt the previous dialogue for which the character responded too

In [27]:
system_promt = """ You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_promt
    prompt += naruto_transcript_df.iloc[ind -1]['line']
    prompt += "\n"
    prompt += naruto_transcript_df.iloc[ind]['line']
    prompts.append(prompt)

In [28]:
print(prompts[0])

 You are naruto from the anime "Naruto". Your Responses should reflect his personalit and speech patterns 
  Oh yeah, Naruto!?
 Where’d you come from, Iruka Sensei!? What are you doing here?


In [29]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,"You are naruto from the anime ""Naruto"". Your ..."
1,"You are naruto from the anime ""Naruto"". Your ..."
2,"You are naruto from the anime ""Naruto"". Your ..."
3,"You are naruto from the anime ""Naruto"". Your ..."
4,"You are naruto from the anime ""Naruto"". Your ..."


In [31]:
dataset = Dataset.from_pandas(df)